# Lab 6: Earth Engine on the Google Cloud Platform

- In this lab we put together the all the skills we have been developing during the course.
- The objective is to achieve an end-to-end satellite data machine learning workflow in a GCP.
- Our workflow is:
    1) Connect the final requied APIs
    2) Increase the GPU quota of the project
    3) Create a sufficently powerful instance
    4) Create and connect data storage
    5) Carry out end-to-end ML
    6) Stop the instance to avoid ongoing costs

## Sources
> https://github.com/google/earthengine-community/blob/master/guides/linked/TF_demo1_keras.ipynb

> https://colab.research.google.com/github/GoogleCloudPlatform/python-docs-samples/blob/main/people-and-planet-ai/land-cover-classification/cloud-tensorflow.ipynb#scrollTo=G7-JZgREZHQK

> https://cloud.google.com/storage/docs/locations

> https://medium.com/@yvanjaquino/earth-engine-authentication-in-google-cloud-python-edition-6df832874f62

> https://cloud.google.com/vertex-ai/docs/workbench/instances/cloud-storage

> https://docs.google.com/presentation/d/1HcbbEnC0wbGfp-d6qXbVkcBryP0acNlBKeMCsDCUXpw/edit#slide=id.g15d38b5130c_10_3150

> https://colab.research.google.com/github/google/earthengine-community/blob/master/guides/linked/Earth_Engine_TensorFlow_AI_Platform.ipynb

> https://github.com/google/earthengine-community/blob/master/guides/linked/Earth_Engine_TensorFlow_tree_counting_model.ipynb

> https://github.com/google/earthengine-community/blob/master/guides/linked/Earth_Engine_TensorFlow_DNN_from_scratch.ipynb

## 1) Connect the APIs
- As ever, make sure you are in the ML project you set up previously (i.e. not your purely GEE projects).
- We already have the Vertex and GEE APIs connected. But there are two more we need for this. Apply them to your project as we did previously:
    - Cloud Run API
    - Dataflow API

## 2) Increase GPU quota
- Go to the IAM and admin tab of your dashboard, the 'Quotas' submenu of that.
- Use the filters slot to get the general GPU limits:-
    - The filter you want to set is GPUs (all regions). Then select the tick-box of the 'Compute Engine API' as below:
![](images/quotafilters.jpg)
- With it selected, click 'Edit Quotas' and set it to 1. With the reason:
    - "I am attending a course at the University of Auckland, 761 Satellite Data and Machine Learning. We are signed up to the Google Education program and we are now at the final stage, whereby we need to spin up an instance we can then do an end-to-end pipeline, that uses a FCNN/CNN for classification tasks. The course ends in 5 weeks, and the instances will not be in use after this."
    
### DO THIS NOW! ASAP
- Mine came through instantly, once approved it can take up to 15mins to appear in your quota allocation.
- However it may take longer... so get it done!
- You get an email when you submit the request and one when it is approved.
- Refresh the quota page of your dashboard to check if it has been increased. Will change from 0 to 1.

For more detail, and screenshots, see this stack overflow post:
>https://stackoverflow.com/questions/53415180/gcp-error-quota-gpus-all-regions-exceeded-limit-0-0-globally


## 3) Set up the instance
Here we need to use the advanced settings in order to have sufficent compute available to us. The required settings are...

- Details:
    - Region: us-central1
        - This is the region that hosts GEE data and therefore is most efficent for the pipeline in this lab. If you were mostly uploading data, then Australia would be the best bet.
        - Sometimes cannot get a instance in a given region as too busy, may have to shop around the regions.
    - Workbench type: Instance
- Environment:
    - Use the latest version
- Machine type: may have to adjust these depending on the region you end up in.
    - n1-highmem-16(16vCPUs, 104 GB RAM)
    - GPU: NVIDIA T4
    - Number of GPUs: 1
    - Install NVIDIA GPU driver automatically for me: Check yes.
    - Leave the defaults as they are for the Shielded VM and Idle shutdown options
- Disks:
    - Data disk size in GB: 200 GB
        - You have seen the size required for many of the prior work that you are looking at using... when doing this for real, consider carefully this size requirement!
        - It is not for long-term storage (that is what your bucket is for), but it is for intermediate processing files, data that has to be loaded into memory, the size of the model etc etc. Anything that is stored or needed during the processing.
    - Otherwise leave as defaults
- Networking:
    - Leave as defaults
- IAM and security:
    - Leave as defaults
- System health:
    - Leave as defaults
    
With this complete, hit 'Create'!
> I also deleted my first test instance, just to free up any quota it was taking up.

## 4) Create a storage bucket
### Key concepts
- You permanently set a geographic location for storing your object data when you create a bucket.
- You cannot change a bucket's location after it's created, but you can move your data to a bucket in a different location.
- You can select from the following location types:
    - A region is a specific geographic place, such as São Paulo.
    - A dual-region is a specific pair of regions, such as Tokyo and Osaka.
    - A multi-region is a large geographic area, such as the United States, that contains two or more geographic places.
- The location type determines how you data is replicated and priced.
- The location information for a bucket is part of the bucket's metadata, which you can view if you have permission to do so.

### Set-up considerations
- Put in the same region that your working instance is going to be in. Minimises data movement time and costs. 
- Putting it in just the one region is cheapest options and perfectly fine for our purposes.
- Prefer locations that are geographically closer to you with low carbon emissions, highlighted with the Leaf icon.

### Get the bucket set up
- Check that are still in the relevant project (remember that assets are not available between projects, only within projects).
- Go to Cloud Storag via the three line option menu top left.
- + Create
- Pick a name that you can type...
    - Mine is ml-satdata-bucket-001, but has to be globally unique!
- Pick region, same as where you have set up the instance.
    - Australia-southeast1 (Sydney) in my case.
- Set to Autoclass
- Leave access control as the defaults
- Leave protection tools as the defaults.
- Make a note of the location pricing at the bottom once you have set all these options. This will give you an idea of the impact on your budget.
    - Mine is at 0.0025 to 0.023 USD per GB/month
    - With 0.025 USD per objects per month additional cost
        - This structure means that they will charge you more if you are storing LOADS of small things. It's a memory optimization thing.
- Confirm that you do not want public access enabled to your data.

Once set up, mine looks like this:
![](images/bucket.jpg)
        


### Data movement into buckets
- Can be done manually via the 'UPLOAD FILES' button the dashboard as in the image above.
- Programmatically from your local to the bucket, using a console/command line/IDE that can see your local file structure and has internet access.
- Programmatically from a source accesible via an internet connection, using the console/notebook/command line with internet access.

### Access your cloud storage in the Vertex AI instance notebook
- Click and drag the left pane, the one with the file names of the notebooks, to the right. This will reveal a menu at the top that includes the 'mount shared storage' button. 
    - Yes, I spent an hour looking for this button even having read the doc linked below. Sigh.
- Mount your bucket:
    - Do so by clicking on the mount storage, 3 lines, icon
    - Enter the name of your bucket, ml-satdata-bucket-001 in my case.
- You should now be able to see the bucket as a file in your instance.

> https://cloud.google.com/vertex-ai/docs/workbench/managed/cloud-storage

Should look something like this once successful (minus my randon code to the right of course):
![](images/bukcetmount.jpg)

## 5) End-to-end ML
We will be following the tutorial and dataset from the source linked at the start of this lab. Modified to our ends! From this point forwards, I provide blocks of code for you to copy into your GCP notebook and modify with your specifics.

- Activate your instance and open up the Jupyter Notebook interface.
- Select the TensorFlow 2-11 notebook

## Introduction
This is an Earth Engine <> TensorFlow demonstration notebook. Specifically, this notebook shows:

1. Exporting training/testing data from Earth Engine in TFRecord format.
2. Preparing the data for use in a TensorFlow model.
3. Training and validating a simple model (Keras Sequential neural network) in TensorFlow.
4. Making predictions on image data exported from Earth Engine in TFRecord format.
5. Ingesting classified image data to Earth Engine in TFRecord format.

This is intended to demonstrate a complete i/o pipeline.

### A quick VertexAI OS diversion...
- What operating system are we on here... ?

In [1]:
#bash...
!pwd

/home/jupyter


## Setup software libraries
Import software libraries and/or authenticate as necessary.

In [3]:
# Supress annoying TensorFlow warnings
TF_CPP_MIN_LOG_LEVEL="2"

In [4]:
# Import the TensorFlow library and check the version.
import tensorflow as tf
print(tf.__version__)

2023-09-18 01:34:27.710268: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-18 01:34:28.640972: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-09-18 01:34:28.641084: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

2.11.0


In [5]:
# We will use the Folium library for visualization. Import the library and check the version.
import folium
print(folium.__version__)

0.14.0


In [6]:
# Numpy is bae
import numpy as np

In [7]:
#!pip install earthengine-api

import ee
from google.auth import compute_engine

scopes = [
    "https://www.google.apis.com/auth/earthengine"
]

credentials = compute_engine.Credentials(scopes=scopes)
ee.Initialize(credentials)

## Define variables
This set of global variables will be used throughout. For this demo, you must have a Cloud Storage bucket into which you can write files.

In [9]:
# Your Earth Engine username.  This is used to import a classified image
# into your Earth Engine assets folder.
USER_NAME = 'dsmi800'

# Cloud Storage bucket into which training, testing and prediction 
# datasets will be written.  You must be able to write into this bucket.
OUTPUT_BUCKET = 'ml-bucket-dsmi800'

# Use Landsat 8 surface reflectance data for predictors.
L8SR = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
# Use these bands for prediction.
BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']

# This is a training/testing dataset of points with known land cover labels.
LABEL_DATA = ee.FeatureCollection('projects/google/demo_landcover_labels')
# The labels, consecutive integer indices starting from zero, are stored in
# this property, set on each point.
LABEL = 'landcover'
# Number of label values, i.e. number of classes in the classification.
N_CLASSES = 3

# These names are used to specify properties in the export of
# training/testing data and to define the mapping between names and data
# when reading into TensorFlow datasets.
FEATURE_NAMES = list(BANDS)
FEATURE_NAMES.append(LABEL)

# File names for the training and testing datasets.  These TFRecord files
# will be exported from Earth Engine into the Cloud Storage bucket.
TRAIN_FILE_PREFIX = 'Training_demo'
TEST_FILE_PREFIX = 'Testing_demo'
file_extension = '.tfrecord.gz'
TRAIN_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TRAIN_FILE_PREFIX + file_extension
TEST_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TEST_FILE_PREFIX + file_extension

# File name for the prediction (image) dataset.  The trained model will read
# this dataset and make predictions in each pixel.
IMAGE_FILE_PREFIX = 'Image_pixel_demo_'

# The output path for the classified image (i.e. predictions) TFRecord file.
OUTPUT_IMAGE_FILE = 'gs://' + OUTPUT_BUCKET + '/Classified_pixel_demo.TFRecord'
# Export imagery in this region.
EXPORT_REGION = ee.Geometry.Rectangle([-122.7, 37.3, -121.8, 38.00])
# The name of the Earth Engine asset to be created by importing
# the classified image from the TFRecord file in Cloud Storage.
OUTPUT_ASSET_ID = 'users/' + USER_NAME + '/Classified_pixel_demo'

## Get Training and Testing data from Earth Engine
To get data for a classification model of three classes (bare, vegetation, water), we need labels and the value of predictor variables for each labeled example. We've already generated some labels in Earth Engine. Specifically, these are visually interpreted points labeled "bare," "vegetation," or "water" for a very simple classification demo (as we did in our random forest lab). For predictor variables, we'll use Landsat 8 surface reflectance imagery, bands 2-7.

> Google hand labelling example: https://code.earthengine.google.com/?scriptPath=Examples%3ADemos%2FClassification

### Prepare Landsat 8 imagery
First, make a cloud-masked median composite of Landsat 8 surface reflectance imagery from 2018. Check the composite by visualizing with folium.

In [11]:
# Cloud masking function.
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).select(BANDS).divide(10000)

# The image input data is a 2018 cloud-masked median composite.
image = L8SR.filterDate('2018-01-01', '2018-12-31').map(maskL8sr).median()

# Use folium to visualize the imagery.
mapid = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
map = folium.Map(location=[38., -122.5])

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data © Google Earth Engine',
    overlay=True,
    name='median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

### Add pixel values of the composite to labeled points
Some training labels have already been collected for you. Load the labeled points from an existing Earth Engine asset. Each point in this table has a property called landcover that stores the label, encoded as an integer. Here we overlay the points on imagery to get predictor variables along with labels.

In [12]:
# Sample the image at the points and add a random column.
sample = image.sampleRegions(
  collection=LABEL_DATA, properties=[LABEL], scale=30).randomColumn()

# Partition the sample approximately 70-30. (No validate, you have seen how to change this in other labs...)
training = sample.filter(ee.Filter.lt('random', 0.7))
testing = sample.filter(ee.Filter.gte('random', 0.7))

# Pprint == 'pretty print'
from pprint import pprint

# Print the first couple points to verify.
pprint({'training': training.first().getInfo()})
pprint({'testing': testing.first().getInfo()})

{'training': {'geometry': None,
              'id': '000066e7d9bc84b3f95d_0',
              'properties': {'B2': 0.049150001257658005,
                             'B3': 0.06965000182390213,
                             'B4': 0.08974999934434891,
                             'B5': 0.1729000061750412,
                             'B6': 0.2125999927520752,
                             'B7': 0.15150000154972076,
                             'landcover': 1,
                             'random': 0.5484198857675888},
              'type': 'Feature'}}
{'testing': {'geometry': None,
             'id': '00009f65e3c9ae02b84e_0',
             'properties': {'B2': 0.05220000073313713,
                            'B3': 0.062049999833106995,
                            'B4': 0.03660000115633011,
                            'B5': 0.01140000019222498,
                            'B6': 0.006800000090152025,
                            'B7': 0.005249999929219484,
                            'landcover'

### Export the training and testing data
Now that there's training and testing data in Earth Engine and you've inspected a couple examples to ensure that the information you need is present, it's time to materialize the datasets in a place where the TensorFlow model has access to them. You can do that by exporting the training and testing datasets to tables in TFRecord format in your Cloud Storage bucket.

> Learn more about TF records: https://www.tensorflow.org/tutorials/load_data/tfrecord

In [13]:
# Make sure you can see the output bucket.  You must have write access.
print('Found Cloud Storage bucket.' if tf.io.gfile.exists('gs://' + OUTPUT_BUCKET) 
    else 'Can not find output Cloud Storage bucket.')

Found Cloud Storage bucket.


Once you've verified the existence of the intended output bucket, run the exports.

In [14]:
# Create the tasks.
training_task = ee.batch.Export.table.toCloudStorage(
  collection=training,
  description='Training Export',
  fileNamePrefix=TRAIN_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

testing_task = ee.batch.Export.table.toCloudStorage(
  collection=testing,
  description='Testing Export',
  fileNamePrefix=TEST_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

In [15]:
# Start the tasks.
training_task.start()
testing_task.start()

### Monitor task progress
You can see all your Earth Engine tasks by listing them. Make sure the training and testing tasks are completed before continuing.

In [16]:
# Print all tasks.
pprint(ee.batch.Task.list())

[<Task A4XN4AFRVCZPDRAKH3UZAXU7 EXPORT_FEATURES: Testing Export (READY)>,
 <Task T3WR2ISKKUBAAANUACXJVMTJ EXPORT_FEATURES: Training Export (READY)>,
 <Task VEP6L2JTWSPDKQOIOHJHMQHI EXPORT_FEATURES: Testing Export (COMPLETED)>,
 <Task JCQAFODL2KTZLAGSMNH33JKE EXPORT_FEATURES: Training Export (COMPLETED)>]


### Check existence of the exported files
If you've seen the status of the export tasks change to COMPLETED, then check for the existence of the files in the output Cloud Storage bucket.

### Check existence of the exported files
If you've seen the status of the export tasks change to COMPLETED, then check for the existence of the files in the output Cloud Storage bucket.

In [17]:
print('Found training file.' if tf.io.gfile.exists(TRAIN_FILE_PATH) 
    else 'No training file found.')
print('Found testing file.' if tf.io.gfile.exists(TEST_FILE_PATH) 
    else 'No testing file found.')

Found training file.
Found testing file.


### Export the imagery
You can also export imagery using TFRecord format. Specifically, export whatever imagery you want to be classified by the trained model into the output Cloud Storage bucket.

In [18]:
# Specify patch and file dimensions.
image_export_options = {
  'patchDimensions': [256, 256],
  'maxFileSize': 104857600,
  'compressed': True
}

# Setup the task.
image_task = ee.batch.Export.image.toCloudStorage(
  image=image,
  description='Image Export',
  fileNamePrefix=IMAGE_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  scale=30,
  fileFormat='TFRecord',
  region=EXPORT_REGION.toGeoJSON()['coordinates'],
  formatOptions=image_export_options,
)

In [19]:
# Start the task.
image_task.start()

In [28]:
# Monitor task progress
pprint(ee.batch.Task.list())

[<Task B7HDISVVYV7BF62DX2JHOS2E EXPORT_IMAGE: Image Export (COMPLETED)>,
 <Task A4XN4AFRVCZPDRAKH3UZAXU7 EXPORT_FEATURES: Testing Export (COMPLETED)>,
 <Task T3WR2ISKKUBAAANUACXJVMTJ EXPORT_FEATURES: Training Export (COMPLETED)>,
 <Task VEP6L2JTWSPDKQOIOHJHMQHI EXPORT_FEATURES: Testing Export (COMPLETED)>,
 <Task JCQAFODL2KTZLAGSMNH33JKE EXPORT_FEATURES: Training Export (COMPLETED)>]


## Data preparation and pre-processing
Read data from the TFRecord file into a tf.data.Dataset. Pre-process the dataset to get it into a suitable format for input to the model.

### Read into a <code>tf.data.Dataset</code>
Here we are going to read a file in Cloud Storage into a <code>tf.data.Dataset</code>. Check that you can read examples from the file. The purpose here is to ensure that we can read from the file without an error. The actual content is not necessarily human readable.

> More on reading data into a <code>dataset</code>: https://www.tensorflow.org/guide/data

In [29]:
# Create a dataset from the TFRecord file in Cloud Storage.
train_dataset = tf.data.TFRecordDataset(TRAIN_FILE_PATH, compression_type='GZIP')

# Print the first record to check.
print(iter(train_dataset).next())

tf.Tensor(b'\nw\n\x0e\n\x02B2\x12\x08\x12\x06\n\x04\x83QI=\n\x0e\n\x02B3\x12\x08\x12\x06\n\x04\xa9\xa4\x8e=\n\x0e\n\x02B4\x12\x08\x12\x06\n\x04\xd9\xce\xb7=\n\x0e\n\x02B5\x12\x08\x12\x06\n\x04\xb3\x0c1>\n\x0e\n\x02B6\x12\x08\x12\x06\n\x04\xd0\xb3Y>\n\x0e\n\x02B7\x12\x08\x12\x06\n\x04\xd1"\x1b>\n\x15\n\tlandcover\x12\x08\x12\x06\n\x04\x00\x00\x80?', shape=(), dtype=string)


2023-09-18 01:47:32.165872: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-18 01:47:32.467833: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-18 01:47:32.468101: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-18 01:47:32.471820: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

### Define the structure of your data
For parsing the exported TFRecord files, <code>featuresDict</code> is a mapping between feature names (<code>featureNames</code> contains the band and label names) and <code>float32</code> <code>tf.io.FixedLenFeature</code> objects. This mapping is necessary for telling TensorFlow how to read data in a TFRecord file into tensors. 

Specifically, <b>all numeric data exported from Earth Engine is exported as <code>float32</code></b>.

(Note: features in the TensorFlow context (i.e. <code>tf.train.Feature</code>) are not to be confused with Earth Engine features (i.e. <code>ee.Feature</code>), where the former is a protocol message type for serialized data input to the model and the latter is a geometry-based geographic data structure.)

In [30]:
# List of fixed-length features, all of which are float32.
columns = [
  tf.io.FixedLenFeature(shape=[1], dtype=tf.float32) for k in FEATURE_NAMES
]

# Dictionary with names as keys, features as values.
features_dict = dict(zip(FEATURE_NAMES, columns))

pprint(features_dict)

{'B2': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B3': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B4': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B5': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B6': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B7': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'landcover': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None)}


### Parse the dataset
Now we need to make a parsing function for the data in the TFRecord files. The data comes in flattened 2D arrays per record and we want to use the first part of the array for input to the model and the last element of the array as the class label. The parsing function reads data from a serialized <code>Example</code> proto into a dictionary in which the keys are the feature names and the values are the tensors storing the value of the features for that example.

> For an explanation on reading <code>Example</code> protos from TFRecord files: https://www.tensorflow.org/tutorials/load_data/tfrecord

In [31]:
def parse_tfrecord(example_proto):
  """The parsing function.

  Read a serialized example into the structure defined by featuresDict.

  Args:
    example_proto: a serialized Example.

  Returns:
    A tuple of the predictors dictionary and the label, cast to an `int32`.
  """
  parsed_features = tf.io.parse_single_example(example_proto, features_dict)
  labels = parsed_features.pop(LABEL)
  return parsed_features, tf.cast(labels, tf.int32)

# Map the function over the dataset.
parsed_dataset = train_dataset.map(parse_tfrecord, num_parallel_calls=5)

# Print the first parsed record to check.
pprint(iter(parsed_dataset).next())

({'B2': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.04915], dtype=float32)>,
  'B3': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.06965], dtype=float32)>,
  'B4': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.08975], dtype=float32)>,
  'B5': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1729], dtype=float32)>,
  'B6': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.2126], dtype=float32)>,
  'B7': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1515], dtype=float32)>},
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>)


Note that each record of the parsed dataset contains a tuple. The first element of the tuple is a dictionary with bands for keys and the numeric value of the bands for values. The second element of the tuple is a class label.

### Create additional features
Another thing we might want to do as part of the input process is to create new features, for example NDVI, a vegetation index computed from reflectance in two spectral bands. Here are some helper functions for that.

In [32]:
def normalized_difference(a, b):
  """Compute normalized difference of two inputs.

  Compute (a - b) / (a + b).  If the denomenator is zero, add a small delta.

  Args:
    a: an input tensor with shape=[1]
    b: an input tensor with shape=[1]

  Returns:
    The normalized difference as a tensor.
  """
  nd = (a - b) / (a + b)
  nd_inf = (a - b) / (a + b + 0.000001)
  return tf.where(tf.math.is_finite(nd), nd, nd_inf)

def add_NDVI(features, label):
  """Add NDVI to the dataset.
  Args:
    features: a dictionary of input tensors keyed by feature name.
    label: the target label

  Returns:
    A tuple of the input dictionary with an NDVI tensor added and the label.
  """
  features['NDVI'] = normalized_difference(features['B5'], features['B4'])
  return features, label

## Model setup
The basic workflow for classification in TensorFlow is:

1. Create the model.
2. Train the model (i.e. <code>fit()</code>).
3. Use the trained model for inference (i.e. <code>predict()</code>).

Here we'll create a Sequential neural network model using Keras. This simple model is inspired by examples in:

> The TensorFlow Get Started tutorial: https://www.tensorflow.org/tutorials/

> The TensorFlow Keras guide: https://www.tensorflow.org/guide/keras#build_a_simple_model

> The Keras <code>Sequential</code> model examples: https://keras.io/guides/sequential_model/

Note that the model used here is purely for demonstration purposes and hasn't gone through any performance tuning.

## Create the Keras model
Before we create the model, there's still a wee bit of pre-processing to get the data into the right input shape and a format that can be used with cross-entropy loss. Specifically, Keras expects a list of inputs and a one-hot vector for the class.

> https://keras.io/api/losses/

> https://www.tensorflow.org/api_docs/python/tf/one_hot

Here we will use a simple neural network model with a 64 node hidden layer, a dropout layer and an output layer. Once the dataset has been prepared, define the model, compile it, fit it to the training data.

> This is the model we will use: https://keras.io/guides/sequential_model/

In [33]:
from tensorflow import keras

# Add NDVI.
input_dataset = parsed_dataset.map(add_NDVI)

# Keras requires inputs as a tuple.  Note that the inputs must be in the
# right shape.  Also note that to use the categorical_crossentropy loss,
# the label needs to be turned into a one-hot vector.
def to_tuple(inputs, label):
  return (tf.transpose(list(inputs.values())),
          tf.one_hot(indices=label, depth=N_CLASSES))

# Map the to_tuple function, shuffle and batch.
input_dataset = input_dataset.map(to_tuple).batch(8)

# Define the layers in the model.
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(64, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(N_CLASSES, activation=tf.nn.softmax)
])

# Compile the model with the specified loss function.
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fit the model to the training data.
model.fit(x=input_dataset, epochs=10)

Epoch 1/10


2023-09-18 01:48:22.782531: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f1704005cb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-18 01:48:22.782573: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla P4, Compute Capability 6.1
2023-09-18 01:48:22.828504: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-18 01:48:23.264590: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


9/9 [==============================] - 3s 42ms/step - loss: 1.0900 - accuracy: 0.3571
Epoch 2/10
9/9 [==============================] - 0s 3ms/step - loss: 1.0452 - accuracy: 0.5000
Epoch 3/10
9/9 [==============================] - 0s 3ms/step - loss: 1.0041 - accuracy: 0.7714
Epoch 4/10
9/9 [==============================] - 0s 3ms/step - loss: 0.9702 - accuracy: 0.9143
Epoch 5/10
9/9 [==============================] - 0s 3ms/step - loss: 0.9278 - accuracy: 0.9143
Epoch 6/10
9/9 [==============================] - 0s 3ms/step - loss: 0.8966 - accuracy: 0.9571
Epoch 7/10
9/9 [==============================] - 0s 3ms/step - loss: 0.8679 - accuracy: 0.9143
Epoch 8/10
9/9 [==============================] - 0s 3ms/step - loss: 0.8232 - accuracy: 0.9286
Epoch 9/10
9/9 [==============================] - 0s 3ms/step - loss: 0.7955 - accuracy: 0.9143
Epoch 10/10
9/9 [==============================] - 0s 3ms/step - loss: 0.7662 - accuracy: 0.9000


### Check model accuracy on the test set
Now that we have a trained model, we can evaluate it using the test dataset. To do that, read and prepare the test dataset in the same way as the training dataset. Here we specify a batch size of 1 so that each example in the test set is used exactly once to compute model accuracy. For model steps, just specify a number larger than the test dataset size (ignore the warning).

In [34]:
test_dataset = (
  tf.data.TFRecordDataset(TEST_FILE_PATH, compression_type='GZIP')
    .map(parse_tfrecord, num_parallel_calls=5)
    .map(add_NDVI)
    .map(to_tuple)
    .batch(1))

model.evaluate(test_dataset)

28/28 [==============================] - 0s 3ms/step - loss: 0.8110 - accuracy: 0.7857


[0.8110493421554565, 0.7857142686843872]

Two metrics of loss and accuracy are output here:
- Loss: monitor it during training to see if it is increasing, decreasing, staying the same. Indicates if have hit a minima/if you are optimised or not.
- Accuracy: monitor it during the test-validate phase as a measure of overall outcome performance.

## Use the trained model to classify an image from Earth Engine
Now it's time to classify the image that was exported from Earth Engine. If the exported image is large, it will be split into multiple TFRecord files in its destination folder. There will also be a JSON sidecar file called "the mixer" that describes the format and georeferencing of the image. Here we will find the image files and the mixer file, getting some info out of the mixer that will be useful during model inference.

### Find the image files and JSON mixer file in Cloud Storage
Use <code>gsutil</code> to locate the files of interest in the output Cloud Storage bucket. Check to make sure your image export task finished before running the following.

In [35]:
# Get a list of all the files in the output bucket.
files_list = !gsutil ls 'gs://'{OUTPUT_BUCKET}
# Get only the files generated by the image export.
exported_files_list = [s for s in files_list if IMAGE_FILE_PREFIX in s]

# Get the list of image files and the JSON mixer file.
image_files_list = []
json_file = None
for f in exported_files_list:
  if f.endswith('.tfrecord.gz'):
    image_files_list.append(f)
  elif f.endswith('.json'):
    json_file = f

# Make sure the files are in the right order.
image_files_list.sort()

pprint(image_files_list)
print(json_file)

['gs://ml-bucket-dsmi800/Image_pixel_demo_00000.tfrecord.gz',
 'gs://ml-bucket-dsmi800/Image_pixel_demo_00001.tfrecord.gz']
gs://ml-bucket-dsmi800/Image_pixel_demo_mixer.json


### Read the JSON mixer file
The mixer contains metadata and georeferencing information for the exported patches, each of which is in a different file. Read the mixer to get some information needed for prediction.

In [36]:
import json

# Load the contents of the mixer file to a JSON object.
json_text = !gsutil cat {json_file}

# Get a single string w/ newlines from the IPython.utils.text.SList
mixer = json.loads(json_text.nlstr)
pprint(mixer)

{'patchDimensions': [256, 256],
 'patchesPerRow': 13,
 'projection': {'affine': {'doubleMatrix': [0.00026949458523585647,
                                            0.0,
                                            -122.70007617412975,
                                            0.0,
                                            -0.00026949458523585647,
                                            38.00089247493765]},
                'crs': 'EPSG:4326'},
 'totalPatches': 130}


### Read the image files into a dataset
You can feed the list of files (<code>imageFilesList</code>) directly to the <code>TFRecordDataset</code> constructor to make a combined dataset on which to perform inference. The input needs to be preprocessed differently than the training and testing. Mainly, this is because the pixels are written into records as patches, we need to read the patches in as one big tensor (one patch for each band), then flatten them into lots of little tensors.

Tensors are the data structure used by machine learning systems. Three properties define a tensor:
- range
- shape
- dType: Here the rank of a tensor refers to the number of axes of the tensor.

> https://furkangulsen.medium.com/what-is-a-tensor-ce8e78835d08

In [39]:
# Get relevant info from the JSON mixer file.
patch_width = mixer['patchDimensions'][0]
patch_height = mixer['patchDimensions'][1]
patches = mixer['totalPatches']
patch_dimensions_flat = [patch_width * patch_height, 1]

# Note that the tensors are in the shape of a patch, one patch for each band.
image_columns = [
  tf.io.FixedLenFeature(shape=patch_dimensions_flat, dtype=tf.float32) 
    for k in BANDS
]

# Parsing dictionary.
image_features_dict = dict(zip(BANDS, image_columns))

# Note that you can make one dataset from many files by specifying a list.
image_dataset = tf.data.TFRecordDataset(image_files_list, compression_type='GZIP')

# Parsing function.
def parse_image(example_proto):
  return tf.io.parse_single_example(example_proto, image_features_dict)

# Parse the data into tensors, one long tensor per patch.
image_dataset = image_dataset.map(parse_image, num_parallel_calls=5)

# Break our long tensors into many little ones.
image_dataset = image_dataset.flat_map(
  lambda features: tf.data.Dataset.from_tensor_slices(features)
)

# Add additional features (NDVI).
image_dataset = image_dataset.map(
  # Add NDVI to a feature that doesn't have a label.
  lambda features: add_NDVI(features, None)[0]
)

# Turn the dictionary in each record into a tuple without a label.
image_dataset = image_dataset.map(
  lambda data_dict: (tf.transpose(list(data_dict.values())), )
)

# Turn each patch into a batch.
image_dataset = image_dataset.batch(patch_width * patch_height)

### Generate predictions for the image pixels
To get predictions in each pixel, run the image dataset through the trained model using <code>model.predict()</code>. Print the first prediction to see that the output is a list of the three class probabilities for each pixel. Running all predictions might take a while.

In [40]:
# Run prediction in batches, with as many steps as there are patches.
predictions = model.predict(image_dataset, steps=patches, verbose=1)

# Note that the predictions come as a numpy array.  Check the first one.
print(predictions[0])

130/130 [==============================] - 192s 1s/step
[[0.2521426  0.6080544  0.13980293]]


### Write the predictions to a TFRecord file
Now that there's a list of class probabilities in <code>predictions</code>, it's time to write them back into a file, optionally including a class label which is simply the index of the maximum probability. We'll write directly from TensorFlow to a file in the output Cloud Storage bucket.

Iterate over the list, compute class label and write the class and the probabilities in patches. Specifically, we need to write the pixels into the file as patches in the same order they came out. The records are written as serialized <code>tf.train.Example</code> protos. This might take a while.

In [41]:
### WARNING! Not tested successfully!! ###

print('Writing to file ' + OUTPUT_IMAGE_FILE)

# Instantiate the writer.
writer = tf.io.TFRecordWriter(OUTPUT_IMAGE_FILE)

# Every patch-worth of predictions we'll dump an example into the output
# file with a single feature that holds our predictions. Since our predictions
# are already in the order of the exported data, the patches we create here
# will also be in the right order.
patch = [[], [], [], []]
cur_patch = 1
for prediction in predictions:
    patch[0].append(tf.argmax(prediction[0]))
    patch[1].append(prediction[0][0])
    patch[2].append(prediction[0][1])
    patch[3].append(prediction[0][2])
    
    # Once we've seen a patches-worth of class_ids...
    if (len(patch[0]) == patch_width * patch_height):
        print('Done with patch ' + str(cur_patch) + ' of ' + str(patches) + '...')
        # Create an example
        example = tf.train.Example(
          features=tf.train.Features(
            feature={
              'prediction': tf.train.Feature(
                  int64_list=tf.train.Int64List(
                      value=patch[0])),
              'bareProb': tf.train.Feature(
                  float_list=tf.train.FloatList(
                      value=patch[1])),
              'vegProb': tf.train.Feature(
                  float_list=tf.train.FloatList(
                      value=patch[2])),
              'waterProb': tf.train.Feature(
                  float_list=tf.train.FloatList(
                      value=patch[3])),
            }
          )
        )
        # Write the example to the file and clear our patch array so it's ready for
        # another batch of class ids
        writer.write(example.SerializeToString())
        patch = [[], [], [], []]
        cur_patch += 1

writer.close()
print('Writing complete!')

Writing to file gs://ml-bucket-dsmi800/Classified_pixel_demo.TFRecord
Done with patch 1 of 130...
Done with patch 2 of 130...
Done with patch 3 of 130...
Done with patch 4 of 130...
Done with patch 5 of 130...
Done with patch 6 of 130...
Done with patch 7 of 130...
Done with patch 8 of 130...
Done with patch 9 of 130...
Done with patch 10 of 130...
Done with patch 11 of 130...
Done with patch 12 of 130...
Done with patch 13 of 130...
Done with patch 14 of 130...
Done with patch 15 of 130...
Done with patch 16 of 130...
Done with patch 17 of 130...
Done with patch 18 of 130...
Done with patch 19 of 130...
Done with patch 20 of 130...
Done with patch 21 of 130...
Done with patch 22 of 130...
Done with patch 23 of 130...
Done with patch 24 of 130...
Done with patch 25 of 130...
Done with patch 26 of 130...
Done with patch 27 of 130...
Done with patch 28 of 130...
Done with patch 29 of 130...
Done with patch 30 of 130...
Done with patch 31 of 130...
Done with patch 32 of 130...
Done with p

## Upload the classifications to an Earth Engine asset
### Verify the existence of the predictions file
At this stage, there should be a predictions TFRecord file sitting in the output Cloud Storage bucket. Use the gsutil command to verify that the predictions image (and associated mixer JSON) exist and have non-zero size.

In [42]:
!gsutil ls -l {OUTPUT_IMAGE_FILE}

 110772220  2023-09-18T03:22:07Z  gs://ml-bucket-dsmi800/Classified_pixel_demo.TFRecord
TOTAL: 1 objects, 110772220 bytes (105.64 MiB)


### Upload the classified image to Earth Engine
Upload the image to Earth Engine directly from the Cloud Storage bucket with the <code>earthengine</code> command. Provide both the image TFRecord file and the JSON file as arguments to <code>earthengine upload</code>.
    
    > https://developers.google.com/earth-engine/command_line#upload

In [43]:
print('Uploading to ' + OUTPUT_ASSET_ID)

# Start the upload.
!earthengine upload image --asset_id={OUTPUT_ASSET_ID} --pyramiding_policy=mode {OUTPUT_IMAGE_FILE} {json_file}

Uploading to users/dsmi800/Classified_pixel_demo
Please authorize access to your Earth Engine account by running

earthengine authenticate

in your command line, and then retry.


### Check the status of the asset ingestion
You can also use the Earth Engine API to check the status of your asset upload. It might take a while. The upload of the image is an asset ingestion task.

In [ ]:
ee.batch.Task.list()

### View the ingested asset
Display the vector of class probabilities as an RGB image with colors corresponding to the probability of bare, vegetation, water in a pixel. Also display the winning class using the same color palette.

In [ ]:
predictions_image = ee.Image(OUTPUT_ASSET_ID)

prediction_vis = {
  'bands': 'prediction',
  'min': 0,
  'max': 2,
  'palette': ['red', 'green', 'blue']
}
probability_vis = {'bands': ['bareProb', 'vegProb', 'waterProb'], 'max': 0.5}

prediction_map_id = predictions_image.getMapId(prediction_vis)
probability_map_id = predictions_image.getMapId(probability_vis)

map = folium.Map(location=[37.6413, -122.2582])
folium.TileLayer(
  tiles=prediction_map_id['tile_fetcher'].url_format,
  attr='Map Data © Google Earth Engine',
  overlay=True,
  name='prediction',
).add_to(map)
folium.TileLayer(
  tiles=probability_map_id['tile_fetcher'].url_format,
  attr='Map Data © Google Earth Engine',
  overlay=True,
  name='probability',
).add_to(map)
map.add_child(folium.LayerControl())
map